## Preparing the dataset for analysis and Model

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import pystan
import pystan_utils
import os

# matplotlib style options
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 10)

In [2]:
# load csv
df = pd.read_csv("SDG_Indicators.csv")
df1 = df.drop(['IndicatorId', 'Series Code', 'Series Type', 'Series Description', 'Parent Country or Area Code',
                'Country or Area Code', 'Country or Area Name', 'LDC', 'LLDC', 'SIDS','Frequency', 'Source type',
                'Location', 'Value type', '1983', 'FN', 'FN.1', 'FN.2', 'FN.3', 'FN.4', 'FN.5', 'FN.6', 'FN.7',
                'FN.8', 'FN.9', 'FN.10', 'FN.11', 'FN.12', 'FN.13', 'FN.14', 'FN.15', 'FN.16', 'FN.17', 'FN.18',
               'FN.19', 'FN.20', 'FN.21','FN.22','FN.23','FN.24',
               'FN.25','FN.26','FN.27','FN.28','FN.29','FN.30','FN.31','FN.32','FN.33','FN.34',
              ], axis=1)
df1.columns

Index(['Goal', 'Target', 'Indicator Ref', 'Indicator Description', 'Age group',
       'Sex', 'Unit', 'Unit multiplier', '1984', '1985', '1986', '1987',
       '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017'],
      dtype='object')

Selecting relevant rows

In [3]:
# Rows
# all age groups
df2 = df1[df1['Age group'].isin(['All age ranges or no breakdown by age'])]
df3 = df2.drop(['Age group'], axis=1)

# all sex
df4 = df3[df3['Sex'].isin(['Both sexes or no breakdown by sex'])]
df5 = df4.drop(['Sex'], axis=1)

# delete rows with no data
df6 = df5[~df5.isnull()]

Use category sheet to find indicators of interest

In [4]:
# import Excel sheet
indicator_categories = pd.read_csv('Dev_Indicators.csv', delimiter=';')
del indicator_categories['Unnamed: 2']
del indicator_categories['Unnamed: 3']
indicator_categories.head()

,Category,Indicator Description
0,----,"Number of verified cases of killing, kidnappin..."
1,E&P (Percent),Number of countries that have national statist...
2,E&P (Percent),Number of countries with a national statistica...
3,E&P (Percent),Energy intensity measured in terms of primary ...
4,E&P (Percent),Annual growth rate of real GDP per capita


In [5]:
ind_cat_joined = indicator_categories.merge(df6, on='Indicator Description')
# delete rows with '----' and '---'
ind_cat_joined_1 = ind_cat_joined[ind_cat_joined.Category != '----']
ind_cat_joined_2 = ind_cat_joined_1[ind_cat_joined_1.Category != '---']
ind_cat_joined_2

,Category,Indicator Description,Goal,Target,Indicator Ref,Unit,Unit multiplier,1984,1985,1986,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
1,E&P (Percent),Number of countries that have national statist...,17,17.18,17.18.2,Number,Units,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.00,NaN
2,E&P (Percent),Number of countries with a national statistica...,17,17.18,17.18.3,Number,Units,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.00,NaN
3,E&P (Percent),Number of countries with a national statistica...,17,17.18,17.18.3,Number,Units,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.00,NaN
4,E&P (Percent),Number of countries with a national statistica...,17,17.18,17.18.3,Number,Units,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.00,NaN
5,E&P (Percent),Number of countries with a national statistica...,17,17.18,17.18.3,Number,Units,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.00,NaN
6,E&P (Percent),Number of countries with a national statistica...,17,17.18,17.18.3,Number,Units,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.00,NaN
7,E&P (Percent),Energy intensity measured in terms of primary ...,7,7.3,7.3.1,Megajoules per USD constant 2011 PPP GDP,Units,NaN,NaN,NaN,...,NaN,NaN,5.95,NaN,NaN,NaN,5.49,NaN,NaN,NaN
8,E&P (Percent),Annual growth rate of real GDP per capita,8,8.1,8.1.1,Percent,Units,NaN,NaN,NaN,...,0.20,-3.24,2.82,1.62,1.01,1.08,1.35,1.45,NaN,NaN
9,E&P (Percent),Annual growth rate of real GDP per employed pe...,8,8.2,8.2.1,Percent,Units,NaN,NaN,NaN,...,1.83,-0.71,4.07,2.66,1.90,1.93,1.80,1.67,1.80,NaN
10,E&P (Percent),Manufacturing value added as a proportion of G...,9,9.2,9.2.1,Percent,Units,NaN,NaN,NaN,...,15.74,14.99,15.81,15.97,15.94,15.99,16.03,16.11,16.16,NaN


### Prepare Population Dataframe

In [6]:
df_population = ind_cat_joined_2[ind_cat_joined_2['Category'].str.contains('Population')]

In [29]:
intyear = list(range(1984, 2018))
#print(intyears)

year = []
for y in intyear:
    year.append(str(y))
    
#print(year)

In [8]:
def popToPercent(unit, n):
    if type(n)==float:
        n2 = n
    elif n.find(',') != -1:
        n = n.replace(',','')
    n2 = float(n)
    if np.isnan(n2):
        return np.nan
    if unit =='Percent':
        return n2
    if unit == 'Per 1,000 population' or 'Per 1,000 uninfected population':
        return n2/10
    if unit == 'Per 100,000 population':
        return n2/1000

In [9]:
df_pop_soon_percent = df_population[~df_population['Unit'].isin(['Number'])]
df_pop_to_percent = df_pop_soon_percent.copy()
for yr in year:
    df_pop_to_percent[yr] = df_pop_to_percent.apply(lambda r: popToPercent(r['Unit'], r[yr]), axis=1)
df_pop_to_percent['Unit'] = 'Percent'

In [10]:
df_pop_percent = df_population[df_population['Unit'].isin(['Percent'])]

In [11]:
df_pop_in_percent = df_pop_percent.append(df_pop_to_percent)

### Economy and Politics

In [12]:
df_ecopol = ind_cat_joined_2[ind_cat_joined_2['Category'].str.contains('E&P')]
df_ecopol.head()

,Category,Indicator Description,Goal,Target,Indicator Ref,Unit,Unit multiplier,1984,1985,1986,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
1,E&P (Percent),Number of countries that have national statist...,17,17.18,17.18.2,Number,Units,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.00,NaN
2,E&P (Percent),Number of countries with a national statistica...,17,17.18,17.18.3,Number,Units,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.00,NaN
3,E&P (Percent),Number of countries with a national statistica...,17,17.18,17.18.3,Number,Units,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.00,NaN
4,E&P (Percent),Number of countries with a national statistica...,17,17.18,17.18.3,Number,Units,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.00,NaN
5,E&P (Percent),Number of countries with a national statistica...,17,17.18,17.18.3,Number,Units,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.00,NaN


In [13]:
# dataframe for indicators which are already expressed in percent
df_ecopol_percent = df_ecopol[df_ecopol['Unit'].isin(['Percent'])]
# dataframe for indicators which need to be converted to percent or USD
df_ecopol_to_percent = df_ecopol[~df_ecopol['Unit'].isin(['Percent'])]

In [14]:
df_ecopol_to_percent_no_countries = df_ecopol_to_percent[df_ecopol_to_percent['Indicator Description'].str.contains('Number of countries')]

for yr in year:
    df_ecopol_to_percent_no_countries[yr] = df_ecopol_to_percent_no_countries.apply(lambda r: float(r[yr])/1.93, axis=1)
df_ecopol_to_percent_no_countries['Unit'] = 'Percent'

df_ecopol_to_percent_no_countries

/home/felix/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/felix/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Category,Indicator Description,Goal,Target,Indicator Ref,Unit,Unit multiplier,1984,1985,1986,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
1,E&P (Percent),Number of countries that have national statist...,17,17.18,17.18.2,Percent,Units,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.170984,NaN
2,E&P (Percent),Number of countries with a national statistica...,17,17.18,17.18.3,Percent,Units,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.145078,NaN
3,E&P (Percent),Number of countries with a national statistica...,17,17.18,17.18.3,Percent,Units,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.326425,NaN
4,E&P (Percent),Number of countries with a national statistica...,17,17.18,17.18.3,Percent,Units,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.626943,NaN
5,E&P (Percent),Number of countries with a national statistica...,17,17.18,17.18.3,Percent,Units,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.808290,NaN
6,E&P (Percent),Number of countries with a national statistica...,17,17.18,17.18.3,Percent,Units,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.968912,NaN
28,E&P (Percent),Number of countries reporting progress in mult...,17,17.16,17.16.1,Percent,Units,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.834197


In [15]:
df_ecopol_percent1 = df_ecopol_percent.append(df_ecopol_to_percent_no_countries)

In [16]:
df_ecopol_to_usd = df_ecopol[df_ecopol['Unit'].str.contains('USD') & ~df_ecopol['Unit'].str.contains('Megajoules')]

In [17]:
def toUSD(unit, multiplier, n):
    if type(n)==float:
        n2 = n
    elif n.find(',') != -1:
        n = n.replace(',','')
    n2 = float(n)
    if np.isnan(n2):
        return np.nan
    if unit=='Constant USD' or unit=='USD':
        if multiplier=='Units':
            return n2
        if multiplier=='Millions':
            return n2/1000000
        if multiplier=='Billions':
            return n2/1000000000
    elif unit=='Per 1,000 USD':
        return n2/1000

In [18]:
for yr in year:
    df_ecopol_to_usd[yr] = df_ecopol_to_usd.apply(lambda r: toUSD(r['Unit'], r['Unit multiplier'], r[yr]), axis=1)
df_ecopol_to_usd

/home/felix/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Category,Indicator Description,Goal,Target,Indicator Ref,Unit,Unit multiplier,1984,1985,1986,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
11,E&P (Percent),Manufacturing value added as a proportion of G...,9,9.2,9.2.1,Constant USD,Units,NaN,NaN,NaN,...,1.490660e+03,1.380790e+03,1.500070e+03,1.543940e+03,1.559710e+03,1.582690e+03,1.611650e+03,1638.060000,1660.800000,NaN
14,E&P (Percent),Direct economic loss in relation to global GDP...,11,11.5,11.5.2,"Per 1,000 USD",Units,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003810,NaN
15,E&P (Percent),Direct economic loss in relation to global GDP...,11,11.5,11.5.2,USD,Units,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,293995.470000,NaN
18,E&P (Percent),Direct economic loss in relation to global GDP...,11,11.5,11.5.2,USD,Millions,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,279.582450,NaN
20,E&P (Percent),"Net official development assistance, total and...",17,17.2,17.2.1,Constant USD,Billions,NaN,NaN,NaN,...,1.122416e-04,1.142808e-04,1.206303e-04,1.196733e-04,1.153076e-04,1.218383e-04,1.239458e-04,0.000132,0.000143,NaN
36,E&P (USD),Dollar value of all resources made available t...,17,17.19,17.19.1,USD,Units,NaN,NaN,NaN,...,2.479536e+08,1.406847e+08,3.519962e+08,4.830671e+08,2.086008e+08,4.431062e+08,3.498444e+08,NaN,NaN,NaN


### Environment

In [19]:
df_environment = ind_cat_joined[ind_cat_joined['Category'].str.contains('Environment')]

In [20]:
df_environment_percent = df_environment[df_environment['Unit'].str.contains('Percent')]
df_environment_not_percent = df_environment[~df_environment['Unit'].str.contains('Percent')]
df_environment_weight_units = df_environment_not_percent[df_environment_not_percent['Unit'].isin(['Metric Tons', 'Micrograms per cubic meter',
                                    'Kilograms', 'Tonne kilometres', 'kg CO2 equivalent per USD1 constant 2005 PPP GDP','Kilograms per constant USD'])]


In [21]:
def convertWeightUnits(unit, multiplier, n):
    if type(n)==float:
        n2 = n
    elif n.find(',') != -1:
        n = n.replace(',','')
    n2 = float(n)
    if np.isnan(n2):
        return np.nan
    elif unit=='Metric Tons' or unit=='Tonne kilometres':
        if multiplier=='Units':
            return n2
        elif multiplier=='Millions':
            return n2*1000000
        elif multiplier=='Thousands':
            return n2*1000
    elif unit=='Kilograms' or unit=='kg CO2 equivalent per USD1 constant 2005 PPP GDP':
        return n2/1000
    elif unit=='Micrograms per cubic meter':
        return n2/1000000000000

In [22]:
def newUnitName(unit):
    if unit=='Metric Tons' or unit=='Tonne kilometres':
        return unit
    elif unit=='Kilograms':
        return 'Metric Tons'
    elif unit=='kg CO2 equivalent per USD1 constant 2005 PPP GDP':
        return 'Metric Tons CO2 equivalent per USD1 constant 2005 PPP GDP'
    elif unit=='Micrograms per cubic meter':
        return 'Metric Tons per cubic meter'

In [23]:
for yr in year:
    df_environment_weight_units[yr] = df_environment_weight_units.apply(lambda r: convertWeightUnits(r['Unit'], r['Unit multiplier'], r[yr]), axis=1)
df_environment_weight_units['Unit multiplier']='Units'
df_environment_weight_units['Unit'] = df_environment_weight_units.apply(lambda r: newUnitName(r['Unit']), axis=1)

/home/felix/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/felix/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/felix/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


Saving all together in one Dataframe

In [24]:
complete_df = df_pop_in_percent.append([df_ecopol_percent1, df_ecopol_to_usd, df_environment_percent, df_environment_weight_units])
complete_df = complete_df.drop(['Category', 'Unit Multiplier'])

In [25]:
complete_df.to_csv('TargetsForAnalysis.csv')